In [1]:
from poisson_fem import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
m = 16
mesh = PoissonFEM.RectangularMesh(np.ones(m)/m)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
mesh.setEssentialBoundary(origin)

TypeError: setEssentialBoundary() missing 1 required positional argument: 'valueFun'

In [ ]:
funSpace = PoissonFEM.FunctionSpace(mesh)

In [ ]:
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)

In [ ]:
# Precomputations
# Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

In [ ]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [ ]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [ ]:
N = 1e4

In [ ]:
# r1 = range(mesh.nCells)
r2 = range(mesh.nEq)
lmbda_np = np.ones(mesh.nCells)
rhs_np = np.ones(mesh.nEq)
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    rhs.setValues(r2, rhs_np)
    K.assemble(lmbda_np)

    # solving
    ksp.setOperators(K.matrix)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

In [ ]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

In [ ]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

In [ ]:
mesh.cells[0].containsEssentialVertex